In [1]:
import cirq
import numpy as np
import scipy
import sympy 
import matplotlib.pyplot as plt
import cirq.experiments.n_qubit_tomography as cen

In [13]:
def expZZ(t):
    ''' return the gate exp( -it * ZZ ) '''
       
    return cirq.ZZPowGate(exponent=2*t/np.pi, global_shift=-0.5)
   
def expX(t):
    ''' return the gate exp( -it * X ) '''
    return cirq.XPowGate(exponent=2*t/np.pi, global_shift=-0.5)

def is_Hermitian(M, rtol = 1e-5, atol = 1e-9):
    return np.allclose(M, np.conjugate(M.T), rtol=rtol, atol=atol)

def is_positive(M, tol = 1e-7):
    s = np.linalg.eigvalsh(M)
    assert (s[0] > -tol)
    for i in range(len(s)):
      if s[i] <= 0:
         s[i] = 1e-12
    return s

In [19]:
# define basic Pauli matrices
s_alpha = [np.array([[1,0],[0,1]],dtype=complex),np.array([[0,1],[1,0]],dtype=complex),np.array([[0,-1j],[1j,0]],dtype=complex),np.array([[1,0],[0,-1]],dtype=complex)]


# define the many-body spin operators
def sp(alpha,n,N):
    Sa = s_alpha[alpha]
    for i in range(n):
        Sa = np.kron(s_alpha[0],Sa)
    for j in range(n+1,N):
        Sa = np.kron(Sa,s_alpha[0])
    return Sa



def magn_exact_diagonalization(L,g,t,Npoints):
  # array containing the magnetization of individual basis states
  magnetization_basis_states = -np.array( [np.sum(2*np.array(cirq.big_endian_int_to_bits(val = n, bit_count = L)) - 1.0)/L for n in range(2**L)] )

  # create the hamiltonian
  hamiltonian = np.zeros((2**L,2**L),dtype=complex)
  for i in range(L):
      hamiltonian += g/2*sp(1,i,L)
      if i != L-1:
          hamiltonian += -1/2*sp(3,i,L)@sp(3,i+1,L)
  
  # diagonalize
  E,V = np.linalg.eig(hamiltonian)

  # time evolve
  magnetization = np.zeros(Npoints)
  initial_state = np.array([int(n==0) for n in range(2**L)])
  overlap = V.transpose().conj() @ initial_state
  for ind,T in enumerate(np.linspace(0,t,Npoints)):
    state_evolved = V @ (np.exp(-1j*T*E) * overlap)
    magnetization[ind] = np.sum(magnetization_basis_states * np.abs(state_evolved)**2)
  
  return magnetization

In [12]:
# System size
L = 10


# System initialization
chain = cirq.GridQubit.rect(1,L)


# Create a circuit
circuit_dummy = cirq.Circuit()
circuit_dummy.append(cirq.I(q) for q in chain)


# Simulate the wave function ...
result_exact = cirq.Simulator().simulate(circuit_dummy)

# ... and extract relevant objects
state = result_exact.state_vector()
state = state/np.linalg.norm(state) # in case not normalized for large system
print(state)
rho = result_exact.density_matrix_of(chain[ round(L/2):L ])

# compute an observable that consists of a sum of Pauli matrices
Paulix = cirq.PauliSum.from_pauli_strings([cirq.X(q) for q in chain])
q_map = result_exact.qubit_map
x_magntization = Paulix.expectation_from_state_vector(state, q_map).real/L

print(x_magntization)

# Perform repeated measurements ...
repetition = 100
circuit_measurement = cirq.Circuit()
circuit_measurement.append(circuit_dummy)
circuit_measurement.append( [cirq.measure(q) for q in chain], strategy = cirq.InsertStrategy.NEW_THEN_INLINE)  
result_measure = cirq.Simulator().run(circuit_measurement, repetitions = repetition)
# ... and extract relevant observables
keys = [f'(0, {i})' for i in range(L)]
counts = result_measure.multi_measurement_histogram(keys = keys)
key0 = tuple( [0] * L )
probability_0 = counts[key0]/repetition # probability_0 = 1 for circuit_dummy



# Tomography experiments
tomo_qubits = chain[round(L/2):L]
tomo_repetition = 1000
exp = cen.StateTomographyExperiment(tomo_qubits)
sam = cirq.Simulator()
probs = cen.get_state_tomography_data(sam, tomo_qubits, circuit_dummy, exp.rot_circuit, exp.rot_sweep, repetitions=tomo_repetition)
tomo_density_matrix = exp.fit_density_matrix(probs)._density_matrix # extract the density matrix from the probabilities (Linear Inversion)



# Also useful: convert numbers into bitstrings and vice versa
bit_string0 = [0] * L
number = cirq.big_endian_bits_to_int(bit_string0)
bit_string1 = cirq.big_endian_int_to_bits(val = number, bit_count = L)
print( bit_string0 == bit_string1 ) # True

[1.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
0.0
True


In [14]:
def evolve_basic(circ,qubits,g,dt):
    """ one step time evolution of qubits in circ by dt 
    through first order approximation"""
   
    tb = -g*dt/2
    Ub = expX(tb)
    
    ta = -dt/2
    Ua = expZZ(ta)
    
    for qubit in qubits:
        circ.append(Ub(qubit)) # e^B = mult_i(exp(-gdt/2*Xi))
    
    for i in range(len(qubits)-1):
        circ.append(Ua(qubits[i],qubits[i+1]))
    
    return circ


def evolve_symmetric(circ,qubits,g,dt):
    
    """ one step time evolution of qubits in circ by dt 
    through first order approximation"""
    
    tb = -g*dt/2
    Ub = expX(tb)
    
    ta = -dt/4
    Ua = expZZ(ta)
    
    for i in range(len(qubits)-1):
        circ.append(Ua(qubits[i],qubits[i+1]))
    
    for qubit in qubits:
        circ.append(Ub(qubit)) # e^B = mult_i(exp(-gdt/2*Xi))
    
    for i in range(len(qubits)-1):
        circ.append(Ua(qubits[i],qubits[i+1]))
    
    return circ

In [ ]:
qubits = cirq.LineQubit.range(10)
Troterization = cirq.Circuit()

